* hparams search:

https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py


In [1]:
# !conda install pandas numpy scipy ipython -y
# !conda install -c maciejkula -c pytorch spotlight -y
# !pip install spotlight

# !conda install pytorch torchvision  cudatoolkit=10.2 -c pytorch --force-reinstall -y

In [2]:
import sys
# sys.path.append("../../")
import os
# import papermill as pm

import pandas as pd
import numpy as np
# import tensorflow as tf
# from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, get_top_k_items, auc

from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, GroupKFold
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from spotlight.cross_validation import random_train_test_split, user_based_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.synthetic import generate_sequential
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.interactions import Interactions
import torch
from spotlight.evaluation import sequence_mrr_score,precision_recall_score
# from spotlight.evaluation import sequence_precision_recall_score # https://maciejkula.github.io/spotlight/evaluation.html#spotlight.evaluation.sequence_precision_recall_score

In [3]:
from spotlight_hyperparams_movielens_sequence import *

NUM_SAMPLES = 15

# LEARNING_RATES = [2e-3, 1e-2, 5 * 1e-2, 1e-1]
# LOSSES = ['bpr', 'hinge', 'adaptive_hinge', 'pointwise']
# BATCH_SIZE = [16,64, 128, 512]
# EMBEDDING_DIM = [ 64, 128, 256]
# # N_ITER = list(range(5, 20))
# N_ITER = [10,20,40]
# L2 =[1e-6, 1e-4,  5e-3,1e-2, 0.0]  ### [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.0] 

random_state = np.random.RandomState(100)

CUDA = (os.environ.get('CUDA') is not None or
        shutil.which('nvidia-smi') is not None)
print(CUDA)
CUDA = False ## pytorch issues

max_sequence_length = 16
min_sequence_length = 4
random_state = np.random.RandomState(100)

True


In [4]:
torch.cuda.is_available()

True

In [5]:
torch.version.cuda

'8.0'

In [6]:
def sequence_precision_recall_score(model, test, k=1, exclude_preceding=False):
    """
    https://maciejkula.github.io/spotlight/_modules/spotlight/evaluation.html#sequence_precision_recall_score
    
    Compute sequence precision and recall scores. Each sequence
    in test is split into two parts: the first part, containing
    all but the last k elements, is used to predict the last k
    elements.

    Parameters
    ----------

    model: fitted instance of a recommender model
        The model to evaluate.
    test: :class:`spotlight.interactions.SequenceInteractions`
        Test interactions.
    exclude_preceding: boolean, optional
        When true, items already present in the sequence will
        be excluded from evaluation.

    Returns
    -------

    mrr scores: numpy array of shape (num_users,)
        Array of MRR scores for each sequence in test.
    """
    sequences = test.sequences[:, :-k]
    targets = test.sequences[:, -k:]
    precision_recalls = []
    for i in range(len(sequences)):
        predictions = -model.predict(sequences[i])
        if exclude_preceding:
            predictions[sequences[i]] = FLOAT_MAX

        predictions = predictions.argsort()[:k]
        precision_recall = _get_precision_recall(predictions, targets[i], k)
        precision_recalls.append(precision_recall)

    precision = np.array(precision_recalls)[:, 0]
    recall = np.array(precision_recalls)[:, 1]
    return precision, recall


def _get_precision_recall(predictions, targets, k):

    predictions = predictions[:k]
    num_hit = len(set(predictions).intersection(set(targets)))

    return float(num_hit) / len(predictions), float(num_hit) / len(targets)


In [7]:
# top k items to recommend
TOP_K = 4

# # Model parameters
# EPOCHS = 10
# BATCH_SIZE =  256#1024

SEED = 0  # Set None for non-deterministic results
random_state = np.random.RandomState(100)

# CUDA = (os.environ.get('CUDA') is not None or
#         shutil.which('nvidia-smi') is not None)
# print(CUDA)
CUDA = False

# user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
# item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

TRAIN_FILE_PATH = os.path.normpath(r"C:\Users\Dan Ofer\Desktop\Stuff\booking_wisdom\booking_train_set.csv") #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"


Spotlight relevant examples:

https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py

https://github.com/maciejkula/spotlight/issues/172
* Dataset creation/loading + get predictions (sequence)

Spotlight hyperparameter tuning:
https://github.com/maciejkula/spotlight/blob/master/examples/movielens_sequence/movielens_sequence.py

In [8]:
# df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE) ## orig 
# print(df.dtypes)
# print(df.shape)
# ## expected columns: ['userID', 'itemID', 'rating', 'timestamp'] (last as float), others as number
# df.head()

In [9]:
# df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE) ## orig 
df = pd.read_csv(TRAIN_FILE_PATH,
                     nrows=140_500,
                     parse_dates=["checkin"],infer_datetime_format=True,usecols=["utrip_id","city_id","checkin"])
user_encoder = LabelEncoder()


## drop consecutive elements (need to do within group, and to order, and what about updating checkout date?)
## 0 cases if we also use checkin or checkout. 
## The "last"  criteria does drop rows! Do we want or not want it???
## We also +- need to merge with total duration data

df["utrip_id"] = user_encoder.fit_transform(df["utrip_id"])
df = df.groupby("utrip_id").filter(lambda x: len(x) >= 4) # keep only trips with at least 4

#####
df["row_num"] = df.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df["utrip_id"].value_counts()
df["total_rows"] = df["utrip_id"].map(utrip_counts)
print("max rows", df["total_rows"].describe())
df["last"] = (df["row_num"] ==df["total_rows"])#.astype(int)
###
df = df.loc[(df[["city_id","utrip_id","last"]].shift() != df[["city_id","utrip_id","last"]]).max(axis=1)]

##########
### truncate longer trips
print(df.shape[0])
# df = df.loc[df["total_rows"]<=max_sequence_length+1]
df = df.groupby("utrip_id").tail(max_sequence_length)
print(df.shape[0])
##########


df = df[["utrip_id","city_id","checkin","last"]]
              
df.columns = ['userID', 'itemID',  'timestamp',"last"] ## no rating col

df.sort_values(["userID","timestamp"],ascending=True,inplace=True) ## reinforce sort order for within groups
# df["rating"] = 1

df = df[['userID', 'itemID',  'timestamp']]
# print(df.dtypes)
print("nunique\n",df.nunique())
print(df.shape)
df.head()

max rows count    140250.000000
mean          6.113198
std           2.828479
min           4.000000
25%           4.000000
50%           5.000000
75%           7.000000
max          48.000000
Name: total_rows, dtype: float64
132353
132212
nunique
 userID       26132
itemID       16875
timestamp      425
dtype: int64
(132212, 3)


,userID,itemID,timestamp
81631,0,64876,2016-06-18
81632,0,55128,2016-06-20
81633,0,9608,2016-06-22
81634,0,31817,2016-06-24
81635,0,36170,2016-06-25


In [10]:
print("unique items (city_id)")
print(df['itemID'].nunique())
df['itemID'].value_counts().describe()

unique items (city_id)
16875


count    16875.000000
mean         7.834785
std         36.513534
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max       1239.000000
Name: itemID, dtype: float64

In [11]:
# ### replace rare variables (under 2 occurrences) with "-1" dummy OR with their hotel country! (as negative mapped number)
city_ids_counts = df["itemID"].value_counts()
print("before uniques",df["itemID"].nunique())
city_ids_counts = city_ids_counts.to_dict()
df["itemID"] = df["itemID"].where(df["itemID"].map(city_ids_counts)>2, -1)

## run encoder on cities (due to negative and to ensure ok ordering for spotlight)
item_encoder = LabelEncoder()
df["itemID"] = user_encoder.fit_transform(df["itemID"])

print("unique items (city_id)")
print(df['itemID'].nunique())
print(df['itemID'].value_counts().describe())

before uniques 16875
unique items (city_id)
5794
count     5794.000000
mean        22.818778
std        191.057396
min          3.000000
25%          4.000000
50%          6.000000
75%         14.000000
max      13819.000000
Name: itemID, dtype: float64


* Create spotlight Interactions dataset
    * https://github.com/maciejkula/spotlight/issues/172
    * Could provide WEIGHTS (more weight to final row in sequence / last) 
    
    * Getting predictions: 
    ```
    predictions = model.predict(ids)

    item_ids= (-predictions).argsort()[:10] # last 10 items
    print(item_ids)
    print(predictions[item_ids])
    ```

In [12]:
# implicit_interactions = Interactions(df['userID'].values, 1+df['itemID'].values, timestamps=df['timestamp'])

# train, test = user_based_train_test_split(implicit_interactions, 0.25) # ,random_state=
# print("train",train)
# print("test",test)

In [13]:
# %%time
# sequential_interaction = train.to_sequence(max_sequence_length=9, min_sequence_length=3)
# print(sequential_interaction)
# test_sequential = test.to_sequence(max_sequence_length=16, min_sequence_length=4)
# print(test_sequential)

In [14]:
%%time
# dataset = get_movielens_dataset('1M')
dataset = Interactions(df['userID'].values, 1+df['itemID'].values, timestamps=df['timestamp'])

train, rest = user_based_train_test_split(dataset,
                                          test_percentage=0.25,
                                          random_state=random_state)
test, validation = user_based_train_test_split(rest,
                                               test_percentage=0.5,
                                               random_state=random_state)
train = train.to_sequence(max_sequence_length=max_sequence_length,
                          min_sequence_length=min_sequence_length)
print("train",train)
test = test.to_sequence(max_sequence_length=max_sequence_length,
                        min_sequence_length=min_sequence_length)
print("test",test)
validation = validation.to_sequence(max_sequence_length=max_sequence_length,
                                    min_sequence_length=min_sequence_length)
print("validation",validation)
mode = "lstm" #sys.argv[1] ## model types

train <Sequence interactions dataset (17751 sequences x 16 sequence length)>
test <Sequence interactions dataset (2904 sequences x 16 sequence length)>
validation <Sequence interactions dataset (2881 sequences x 16 sequence length)>
Wall time: 103 ms


In [15]:
%%time
run(train, test, validation, random_state, "lstm")

# Best lstm result: {'test_mrr': 0.2304, 'validation_mrr': 0.256,
# 'n_iter': 5, 'loss': 'adaptive_hinge', 'learning_rate': 0.05, 'l2': 0.0001, 'embedding_dim': 64, 'batch_size': 512}


# Evaluating {'n_iter': 20, 'loss': 'adaptive_hinge', 'learning_rate': 0.01, 'l2': 0.0001, 'embedding_dim': 128, 'batch_size': 128}
# Test MRR 0.269 val MRR 0.2738\


# {'n_iter': 40, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 1e-08, 'embedding_dim': 128, 'batch_size': 128}
# Test MRR 0.2826 val MRR 0.2798



# ### less data samples :
# Evaluating {'num_negative_samples': 10, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.005, 'l2': 0.0, 'embedding_dim': 256, 'batch_size': 128}
# Test MRR 0.306 val MRR 0.311

# Evaluating {'num_negative_samples': 10, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.001, 'l2': 0.0, 'embedding_dim': 128, 'batch_size': 128}
# Test MRR 0.3078 val MRR 0.308


## another orund, smallishj samploes 
# Evaluating {'num_negative_samples': 20, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 0.0, 'embedding_dim': 256, 'batch_size': 256}
# Test MRR 0.330 val MRR 0.32771

Best lstm result: {'test_mrr': 0.330401685693058, 'validation_mrr': 0.32771968962309106, 'num_negative_samples': 20, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 0.0, 'embedding_dim': 256, 'batch_size': 256}
Evaluating {'num_negative_samples': 15, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 0.0, 'embedding_dim': 128, 'batch_size': 256}
Test MRR 0.32839684116591195 val MRR 0.33631885048360144
Evaluating {'num_negative_samples': 30, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 0.0, 'embedding_dim': 128, 'batch_size': 256}
Test MRR 0.3325081835544003 val MRR 0.3482472573988587
Evaluating {'num_negative_samples': 60, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 0.0, 'embedding_dim': 128, 'batch_size': 256}
Test MRR 0.3464995984126844 val MRR 0.3605211799916017
Evaluating {'num_negative_samples': 100, 'n_iter': 50, 'loss': 'adaptive_hinge', 'learning_rate': 0.002, 'l2': 0.0, 'embedding_dim':

KeyboardInterrupt: 

In [16]:
%%time
# run(train, test, validation, random_state, model_type="pooling")

# # Best pooling result: {'test_mrr': 0.28493, 'validation_mrr': 0.222,
# # 'n_iter': 15, 'loss': 'hinge', 'learning_rate': 0.01, 'l2': 0.0, 'embedding_dim': 64, 'batch_size': 16}

# Best pooling result: {'test_mrr': 0.2849, 'validation_mrr': 0.22,
#                       'n_iter': 15, 'loss': 'hinge', 'learning_rate': 0.01, 'l2': 0.0, 'embedding_dim': 64, 'batch_size': 16}

Wall time: 0 ns
